# Code ideas

install stuff with

    import Pkg
    Pkg.add("Plots")

## Initialization

In [ ]:
include("initializer.jl")
using Random
import StaticArrays

In [16]:
rng = Random.MersenneTwister(0)
num = 5
dims = 3
positions = initialize(num, dims, rng)
temp = copy(positions)

3×5 Matrix{Float64}:
  0.647295  -0.645342  -0.915397  0.946433   -0.479928
  0.820713  -0.44224   -0.863461  0.171623    0.820093
 -0.670868  -0.593047  -0.276343  0.0785786  -0.665928

## Particles

We want to save some values that are often calculated for each set of positions.

In [ ]:
include("particles.jl")

In [ ]:
particles = Particles(positions)

## Wavefunctions

Use seperate structs for seperate types of wavefunctions and use multiple dispatch to use the same set of function names for all wavefunctions.

In [99]:
include("wavefunctions.jl")

paramDer (generic function with 1 method)

In [12]:
wf = SimpleGaussian(0.4, [1, 1, 1])

SimpleGaussian(0.4, [1.0, 1.0, 1.0])

In [19]:
@time kinetic(positions, wf, temp)

  0.000024 seconds (9 allocations: 256 bytes)


3.937932009660991

In [102]:
@time ratio(positions, 1, positions[:, 2], wf)

  0.000019 seconds (7 allocations: 432 bytes)


0.7933021149451239

In [98]:
@time ratio2(positions, 1, positions[:, 2], wf)

  0.000020 seconds (7 allocations: 432 bytes)


0.7933021149451239

In [ ]:
0.000018 seconds (7 allocations: 432 bytes)

In [95]:
function ratio2(positions, p1, old_pos, wf::SimpleGaussian)::Float64
    """
    Old wavefunc value term: exp(-alpha * old_r2)
    New wavefunc value term: exp(-alpha * r2)
    All other terms are the same and cancel. Since they are both exponentials, we can subtract the exponents
    """
    #r2 = sum(particles.positions[p1].^2 .* wf.HOshape)
    #old_r2 = sum(old_pos.^2 .* wf.HOshape)
    #return exp.(wf.alpha * (old_r2 - r2))
    temp = old_pos.^2
    temp .= temp .- positions[:, p1].^2
    temp .= temp .* wf.HOshape
    return exp(wf.alpha * sum(temp))
end

ratio2 (generic function with 1 method)

## Hamiltonians

Do the same thing as for wavefunctions.

In [ ]:
include("hamiltonians.jl")

In [ ]:
ham = HarmonicOscillator(0.6, [1, 1, 1])

In [ ]:
@time potential(particles, ham)

## Sampler

The sampler should be a mutable struct with any shape for the parameter derivative.

In [ ]:
include("sampler.jl")

In [ ]:
samples = Samples()

In [ ]:
sample!(samples, particles, wf, ham)

## Parallelization

In [103]:
include("initializer.jl")
include("wavefunctions.jl")
include("hamiltonians.jl")
include("sampler.jl")
include("metropolis.jl")
include("particles.jl")

import Random

In [104]:
function runn2(positions, wf, rng, samples, ham)
    temp = copy(positions)
    for j in 1:833333
        metro_step!(positions, wf, rng, 0.01)
        sample!(samples, positions, wf, ham, temp)
    end
end

runn2 (generic function with 1 method)

In [105]:
function runn()
    wf = SimpleGaussian(0.4, [1, 1, 1])
    ham = HarmonicOscillator(0.6, [1, 1, 1])

    nthreads = Threads.nthreads()
    rng_threads = [Random.MersenneTwister() for i in 1:nthreads]
    samples_threads = [Samples() for i in 1:nthreads]

    Threads.@threads for i = 1:nthreads
        rng = rng_threads[i]
        samples = samples_threads[i]
        positions = initialize(10, 3, rng)
        
        runn2(positions, wf, rng, samples, ham)
    end
end

runn (generic function with 1 method)

In [106]:
@time runn()

  9.001924 seconds (220.50 M allocations: 9.266 GiB, 26.12% gc time, 0.00% compilation time)


In [ ]:
cutting up ratio
9.001924 seconds (220.50 M allocations: 9.266 GiB, 26.12% gc time, 0.00% compilation time)

In [ ]:
cutting up kinetic
10.125701 seconds (330.04 M allocations: 12.669 GiB, 31.91% gc time, 0.46% compilation time)

In [ ]:
sending in temp
10.888009 seconds (351.21 M allocations: 13.924 GiB, 32.91% gc time, 3.20% compilation time)

In [ ]:
positions are back
17.690428 seconds (550.00 M allocations: 27.866 GiB, 42.62% gc time)

In [ ]:
20.891537 seconds (650.90 M allocations: 18.959 GiB, 24.54% gc time, 0.00% compilation time)

In [ ]:
20.376229 seconds (671.04 M allocations: 20.160 GiB, 25.68% gc time, 0.02% compilation time)

In [ ]:
18.604906 seconds (670.62 M allocations: 20.136 GiB, 27.61% gc time)

## SArray

The particles are a dim x num [Static Array](https://github.com/JuliaArrays/StaticArrays.jl). If there are more than 100 elements, i.e 34 particles in 3D or 100 particles in 1D, normal arrays should be used instead. This should be benchmarked. Small arrays will be the focus of the masters anyway.

The potential shape should maybe be an SVector.

In [ ]:
using LinearAlgebra
using StaticArrays

# Use the convenience constructor type `SA` to create vectors and matrices
SA[1, 2, 3]     isa SVector{3,Int}
SA_F64[1, 2, 3] isa SVector{3,Float64}
SA_F32[1, 2, 3] isa SVector{3,Float32}
SA[1 2; 3 4]     isa SMatrix{2,2,Int}
SA_F64[1 2; 3 4] isa SMatrix{2,2,Float64}

# Create an SVector using various forms, using constructors, functions or macros
v1 = SVector(1, 2, 3)
v1.data === (1, 2, 3) # SVector uses a tuple for internal storage
v2 = SVector{3,Float64}(1, 2, 3) # length 3, eltype Float64
v3 = @SVector [1, 2, 3]
v4 = @SVector [i^2 for i = 1:10] # arbitrary comprehensions (range is evaluated at global scope)
v5 = zeros(SVector{3}) # defaults to Float64
v6 = @SVector zeros(3)
v7 = SVector{3}([1, 2, 3]) # Array conversions must specify size

In [ ]:
import StaticArrays as sa

sa.SA[1, 2, 3]

a = sa.SMatrix{3, 2, Float64}([[1, 2, 3] [4,5,6]])

## Autograd

Maybe use [ReverseDiff](https://github.com/JuliaDiff/ReverseDiff.jl) or [ForwardDiff](https://github.com/JuliaDiff/ForwardDiff.jl)? Which is faster depends on the number of input and output parameters and the number of operations and whether they are matrix operations. Some benchmarking is required.

In [ ]:
import ForwardDiff

In [ ]:
f(x::Vector) = sum(sin, x) + prod(tan, x) * sum(sqrt, x);

x = rand(5)

g = x -> ForwardDiff.gradient(f, x); # g = ∇f

g(x)

ForwardDiff.hessian(f, x)